In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import h5py  
import numpy as np
import os 
from scipy.misc import imresize
import skvideo.io
import cv2
import random
import soundfile as sf

import category_getter

In [3]:
# def process_data(data_folder = '/media/jeff/Backup/CS598PS/train_nice_6340/'):
data_folder = '/media/jeff/Backup/CS598PS/test/'
filenames = []
videos = []
sounds = []
labels_lst = ['Animal','Channel, environment and background','Human sounds','Music',
             'Natural sounds','Sounds of things','Source-ambiguous sounds']
# cg = category_getter.CategoryGetter("unbalanced_train_segments.csv")
for folder in os.listdir(data_folder): 
    try:
        for filename in os.listdir(os.path.join(data_folder, folder)):
            if '.mp4' in filename: # video files
                clip_name = filename.split('.mp4')[0]
                if os.path.exists(os.path.join(data_folder,folder, clip_name + '.flac')):
                    data, samplerate = sf.read(os.path.join(data_folder,folder, clip_name + '.flac'))
                    if samplerate == 44100 and len(data)==441000: # make sure audio sample rate is consistent
                        data = data[::2] # Subsample to 22050 hertz
                        cap = cv2.VideoCapture(os.path.join(data_folder,folder,filename))
                        frames = []
                        ret = True # flag for remaining frames in cap
                        while(ret):
                            ret, frame = cap.read()
                            if ret:
                                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # convert rgb to grayscale
                                frames.append(imresize(frame,(256,256))) # resize to 256x256
                        if len(frames)>100: # guarantee video has at least 100 frames for 10 seconds
                            frames = np.array([frames[int(i*len(frames)/100.)] for i in range(100)]) # subsample 100 frames
                            youtube_id = '_'.join(filename.split("_")[:-2])
#                             labels.append([labels_lst.index(cg.ontology.get_record_for_id(id)["name"]) for id in cg.get_general_categories_for_video(youtube_id)])
                            videos.append(frames)
                            sounds.append(data)
                            filenames.append(filename)
    except Exception as e:
        print(e)
        print(filename)

videos = np.array(videos)
sounds = np.array(sounds)
# labels = np.array(labels)
filenames = np.array(filenames)
# return videos, sounds, filenames

# videos_train, sounds_train = process_data('/media/jeff/Backup/CS598PS/train_nice_6340/')
# videos_test, sounds_test = process_data('/media/jeff/Backup/CS598PS/test/')
# print(videos_train.shape, sounds_train.shape)
# print(videos_test.shape, sounds_test.shape)

ZeroDivisionError: division by zero

In [4]:
print(videos.shape, filenames.shape)

(207, 100, 256, 256) (207,)


In [5]:
print(filenames)

['9pgSPM4_wjM_540000_550000.mp4' 'c4ftOyl3j_Y_40000_50000.mp4'
 'jo-MHev9r9E_380000_390000.mp4' 'U81BlyV26gw_70000_80000.mp4'
 'j2R1zurR39E_30000_40000.mp4' 'xqILXIFBZfQ_30000_40000.mp4'
 'iE8pSulXDuc_90000_100000.mp4' '-5CGQGSFGyg_60000_70000.mp4'
 'wm3eNSHIyu0_30000_40000.mp4' 'DHcsdmKkoWA_80000_90000.mp4'
 '_VmxkpgiURA_80000_90000.mp4' 'LCJblaUkkfc_170000_180000.mp4'
 'ykgfwnVjv8Y_0_10000.mp4' '4UkP1uB1nLU_0_10000.mp4'
 'rDjjLTi8YjQ_30000_40000.mp4' 'sYIymaJi6tc_90000_100000.mp4'
 'wEOh0U-5tgI_30000_40000.mp4' '0CTgEP0SIVg_10000_20000.mp4'
 'FJmnucB-Wik_30000_40000.mp4' 'GCo2uCLXvhk_30000_40000.mp4'
 'qqdqstHQ5e0_380000_390000.mp4' 'S_Z7o4OmU30_30000_40000.mp4'
 'dJJ-duXBnXI_30000_40000.mp4' 'S__C4GPMIsg_30000_40000.mp4'
 'Sgdq22nyfTM_240000_250000.mp4' 'a_HzXa0YVIo_170000_180000.mp4'
 'HI1_HYH74h4_310000_320000.mp4' 'uGMTZ9PAXkA_100000_110000.mp4'
 'x-2Abohj8VY_30000_40000.mp4' 'CwgPdC20hew_310000_320000.mp4'
 'Ml-jhHJA7s0_180000_190000.mp4' 'fIer0uCLqSM_30000_40000.mp4'
 'Ukb1BnEG

In [11]:
# dataset = h5py.File('/media/jeff/Backup/CS598PS/data_nice_2597_old.h5', 'r')
# videos_train = np.array(dataset['videos_train'])
# sounds_train = np.array(dataset['sounds_train'])
hf.close()
hf = h5py.File('/media/jeff/Backup/CS598PS/test.h5', 'w')
# hf.create_dataset('videos_train', data=videos_train)
# hf.create_dataset('sounds_train', data=sounds_train)
hf.create_dataset('videos_test', data=videos)
hf.create_dataset('sounds_test', data=sounds)
# hf.create_dataset('filenames', data=filenames)
hf.close()

In [10]:
filenames.shape

(807,)

In [12]:
np.save('/media/jeff/Backup/CS598PS/test_filenames.npy', filenames)

In [37]:
filenames

array(['-0SblabL0SI_220000_230000.mp4', '-BbV8M6zlwM_60000_70000.mp4',
       '0-b27jD0ziU_210000_220000.mp4', ...,
       'ODU3EguaDyE_40000_50000.mp4', 'Z3fuzmCzAfk_30000_40000.mp4',
       'j9puj2X76-A_130000_140000.mp4'], dtype='<U29')

In [32]:
data_folder = '/media/jeff/Backup/CS598PS/train_nice_6340/'
filenames = []
for folder in os.listdir(data_folder): 
    try:
        for filename in os.listdir(os.path.join(data_folder, folder)):
            filenames.append(filename)
    except:
        pass
cg = category_getter.CategoryGetter("unbalanced_train_segments.csv") # For the train_N is from "balanced_train_segments.csv"
# for the train_nice it's from "unbalanced_train_segments.csv"
for file_name in filenames:
    youtube_id = '_'.join(file_name.split("_")[:-2])
    print([labels_lst.index(cg.ontology.get_record_for_id(id)["name"]) for id in cg.get_general_categories_for_video(youtube_id)])

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 5, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0, 4]
[0, 3]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]


[0]
[0]
[0]
[0, 3]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0]
[0, 3, 1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 4]
[0]
[0]
[0]
[0]
[0, 4]
[0]
[0, 3]
[0]
[0, 4]
[0]
[0]
[0, 1]
[0]
[0]
[0, 1]
[0]
[0]
[0, 3, 4]
[0]
[0, 3]
[0]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0, 1]
[0]
[0]
[0]
[0]
[0, 4]
[0, 4]
[0]
[0]
[0, 4]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 3]
[0, 3]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 1]
[0, 1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[3, 1]
[3, 1]
[1]
[1]
[1]
[1]
[3, 1]
[3, 1]
[3, 1]
[3, 1]
[1]
[1]
[1]
[1]
[3, 1]
[3, 1]
[1]
[1]
[3, 1]
[3, 1]
[3, 1]
[3, 1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0

In [2]:
class AudioDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train, frames_len=40, transform=None, h5_file='data.h5', transform_label=None):
        """
        Args:
            train (bool): Whether or not to use training data
            frames (int): Number of video frames per video sample
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = train
        self.transform = transform
        self.frames_len = frames_len
        
        dataset = h5py.File(h5_file)
        self.videos_train = np.array(dataset['videos_train'])
        self.sounds_train = np.array(dataset['sounds_train'])
        self.videos_test = np.array(dataset['videos_test'])
        self.sounds_test = np.array(dataset['sounds_test'])
        dataset.close()
        
    def __len__(self):
        if self.train:
            return len(self.videos_train)
        return len(self.videos_train)

    def __getitem__(self, idx):
        if self.train:
            image = self.videos_train[idx]
            audio = self.sounds_train[idx]
        else:
            image = self.videos_test[idx]
            audio = self.sounds_test[idx]

        # Randomly sample 4 seconds from 10 second clip
        start = random.randint(0, 100-self.frames_len) # Start frame
        new_image = np.zeros((self.frames_len,256,256,1), dtype=np.uint8)
        for i in range(self.frames_len):
            new_image[i] = np.expand_dims(image[start+i],2)
        
        # Randomly align or misalign audio sample
        if random.random() < 0.5: # align
            audio = audio[int(start*220500/100.0):int(start*220500/100.0)+88200]
            label = 0
        else: # misalign
            shift = random.randint(20, 60) # frame shift amount
            if random.random() < 0.5: # Add shift
                start = np.clip(start-shift, 0, 100-self.frames_len)
            else: # Subtract shift
                start = np.clip(start+shift, 0, 100-self.frames_len)
            audio = audio[int(start*220500/100.0):int(start*220500/100.0)+88200]
            label = 1
            
        transform_image = np.zeros((self.frames_len,1,224,224), dtype=np.uint8)
        if self.transform:
            for i in range(self.frames_len):
                transform_image[i] = self.transform(new_image[i]) # Transform image frames
            
        return (new_image, audio, label)

In [3]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = AudioDataset(train=True,transform=transform)
test_dataset = AudioDataset(train=False,transform=transforms.ToTensor())

# # Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size
                                           =5, 
                                           shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=5, 
                                           shuffle=False, num_workers=4)

In [4]:
for images, sounds, labels in train_loader:
    print(images.shape)
    print(sounds.shape)
    print(labels.shape)

torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 256, 1])
torch.Size([5, 88200, 2])
torch.Size([5])
torch.Size([5, 40, 256, 2